In [3]:
import sys
!{sys.executable} -m pip install faiss-cpu


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import faiss



In [5]:
tennessee = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Tennesse.csv")
georgia = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Georgia.csv")
ohio = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Ohio.csv")
california = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\California.csv")
florida = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Florida.csv")
washington = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Washington.csv")
colorado = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Colorado.csv")
massachusetts = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Massachusetts.csv")
kansas = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Kansas.csv")
north_carolina = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\North_Carolina.csv")
missouri = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Missouri.csv")
minnesota = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Minnesota.csv")
pennsylvania = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Pennsylvania.csv")
utah = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Utah.csv")
illinois = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Illinois.csv")
texas = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Texas.csv")
oregon = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\Oregon.csv")
new_york = pd.read_csv(r"C:\Users\jessi\Desktop\Proyecto Final Henry\New_York.csv")

dataframes = [tennessee, georgia, ohio, california, florida, washington, colorado, massachusetts, kansas, north_carolina, missouri, minnesota, pennsylvania, utah, illinois, texas, oregon, new_york]

estados = pd.concat(dataframes, ignore_index=True)

In [6]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer



In [7]:
estados = estados.dropna()


In [53]:
estados['combined_features'] = estados['nombre_local'].astype(str) + ' ' + estados['ciudad'].astype(str).fillna('') + ' ' + estados['review_sentiment'].astype(str).fillna('')

# Muestra de datos para evitar problemas de memoria
sample_size = 2000  # Aumentar el tamaño de la muestra si es necesario
df_sampled = estados.sample(n=sample_size, random_state=42)

# Vectorización TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_sampled['combined_features'])

# Cálculo de la similitud del coseno
cosine_sim_sampled = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim_sampled_df = pd.DataFrame(cosine_sim_sampled, index=df_sampled['nombre_local'], columns=df_sampled['nombre_local'])

def recommend_restaurant(restaurante, ciudad):
    restaurante = restaurante.strip()  

    if restaurante not in cosine_sim_sampled_df.index:
        return f"Restaurant named '{restaurante}' not found in the dataset."

    restaurantes_ciudad = df_sampled[df_sampled['ciudad'].str.lower() == ciudad.lower()]

    # Verificar que hay restaurantes en la ciudad especificada
    if restaurantes_ciudad.empty:
        return f"No restaurants found in the city '{ciudad}'."

    sim_scores = cosine_sim_sampled_df.loc[restaurante, restaurantes_ciudad['nombre_local']]

    sim_scores = sim_scores.sort_values(ascending=False)
    top_similar_restaurants = sim_scores.index[sim_scores.index != restaurante]

    recommended_restaurants = restaurantes_ciudad[restaurantes_ciudad['nombre_local'].isin(top_similar_restaurants)][['nombre_local', 'avg_rating', 'direccion', 'ciudad']]
    recommended_restaurants = recommended_restaurants.drop_duplicates(subset=['nombre_local']).iloc[:5]


    return recommended_restaurants 

# Ejemplo de uso
resultado = recommend_restaurant("Mezcaleria Las Flores", "Chicago")
print(resultado)

                     nombre_local  avg_rating              direccion   ciudad
265577  Vapiano Chicago Riverside         4.3  222 S Riverside Plaza  Chicago
240067       Devil Dawgs on State         4.4         767 S State St  Chicago
258564                    Kaathis         3.9     2273 N Lincoln Ave  Chicago
232663              Wake ‘n Bacon         4.1      420 W Belmont Ave  Chicago
268318    Little Bucharest Bistro         4.6      3661 N Elston Ave  Chicago
